# Gender Prediction by Speech Content

In [2]:
import joblib
import re
import string

import numpy as np
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import SnowballStemmer

In [3]:
import nltk
from nltk.corpus import stopwords

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
stop_words = list(set(stopwords.words('german')))

In [6]:
# load data
df = pd.read_pickle('C:/Users/Ana/OneDrive - Hochschule Düsseldorf/MA/data_topics.pkl')

df.head()

,text,date,legislative_period,speaker_name,speaker_gender,speaker_role,speaker_party,comments,id,speaker_id,speech_length,topic_distribution
0,Sehr geehrter Herr Alterspräsident! So muss ...,2021-10-26,20,Gabriele Katzmarek,female,None,SPD,[(Beifall bei der SPD sowie bei Abgeordneten d...,SP-2021-0,1299,2593,{68: 0.35882655}
1,Herr Präsident! Liebe Kolleginnen und Kolleg...,2021-10-26,20,Stefan Müller,male,None,CDU/CSU,[(Beifall bei der CDU/CSU sowie bei Abgeordnet...,SP-2021-1,4028,3796,{68: 0.46776655}
2,Sehr geehrter Herr Alterspräsident Wolfgang ...,2021-10-26,20,Britta Haßelmann,female,None,BÜNDNIS 90/DIE GRÜNEN,[(Beifall beim BÜNDNIS 90/DIE GRÜNEN und bei d...,SP-2021-2,451,4301,{68: 0.26328164}
3,Sehr geehrter Herr Präsident! Meine lieben K...,2021-10-26,20,Marco Buschmann,male,None,FDP,"[(Jan Korte [DIE LINKE]: Oder Jugendweihe!), (...",SP-2021-3,3083,4555,{}
4,Herr Alterspräsident! Lassen Sie mich zunäch...,2021-10-26,20,Stephan Brandner,male,None,AfD,"[(Beifall bei der AfD), (Beifall bei Abgeordne...",SP-2021-4,4055,5996,{68: 0.3120498}


In [7]:
df = df[["speaker_gender", "text"]]
df.head()

,speaker_gender,text
0,female,Sehr geehrter Herr Alterspräsident! So muss ...
1,male,Herr Präsident! Liebe Kolleginnen und Kolleg...
2,female,Sehr geehrter Herr Alterspräsident Wolfgang ...
3,male,Sehr geehrter Herr Präsident! Meine lieben K...
4,male,Herr Alterspräsident! Lassen Sie mich zunäch...


In [8]:
def process_text(text):
    text = str(text).lower()
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", " ", text
    )
    text = " ".join(text.split())

    return text

In [9]:
df["clean_text"] = df.text.map(process_text)

In [10]:
df.head()

,speaker_gender,text,clean_text
0,female,Sehr geehrter Herr Alterspräsident! So muss ...,sehr geehrter herr alterspräsident so muss ich...
1,male,Herr Präsident! Liebe Kolleginnen und Kolleg...,herr präsident liebe kolleginnen und kollegen ...
2,female,Sehr geehrter Herr Alterspräsident Wolfgang ...,sehr geehrter herr alterspräsident wolfgang sc...
3,male,Sehr geehrter Herr Präsident! Meine lieben K...,sehr geehrter herr präsident meine lieben koll...
4,male,Herr Alterspräsident! Lassen Sie mich zunäch...,herr alterspräsident lassen sie mich zunächst ...


In [11]:
len(df)

971264

In [30]:
df_train, df_test = train_test_split(df, test_size=0.20, stratify=df.speaker_gender, random_state=0)

In [62]:
vec = CountVectorizer(
    ngram_range=(1, 3), 
    stop_words=stop_words,
)  

X_train = vec.fit_transform(df_train.clean_text)
X_test = vec.transform(df_test.clean_text)

y_train = df_train.speaker_gender
y_test = df_test.speaker_gender

In [18]:
# split data for testing
#specify number of rows in each chunk
n=100000

#split DataFrame into chunks
splitted_df = [df[i:i+n] for i in range(0,len(df),n)]

In [24]:
small_df = splitted_df[0]
small_df

,speaker_gender,text,clean_text
0,female,Sehr geehrter Herr Alterspräsident! So muss ...,sehr geehrter herr alterspräsident so muss ich...
1,male,Herr Präsident! Liebe Kolleginnen und Kolleg...,herr präsident liebe kolleginnen und kollegen ...
2,female,Sehr geehrter Herr Alterspräsident Wolfgang ...,sehr geehrter herr alterspräsident wolfgang sc...
3,male,Sehr geehrter Herr Präsident! Meine lieben K...,sehr geehrter herr präsident meine lieben koll...
4,male,Herr Alterspräsident! Lassen Sie mich zunäch...,herr alterspräsident lassen sie mich zunächst ...
...,...,...,...
100010,male,Eine Zusatzfrage des Abgeordneten Dr. Mommer!,eine zusatzfrage des abgeordneten dr mommer
100011,male,"Frau Minister, Sie verstehen sicher, daß die ...",frau minister sie verstehen sicher daß die bet...
100012,female,"Herr Kollege Mommer, ich bekomme zahlreiche B...",herr kollege mommer ich bekomme zahlreiche bri...
100013,male,Eine Zusatzfrage des Abgeordneten Dr. Schäfer!,eine zusatzfrage des abgeordneten dr schäfer


In [25]:
small_df_train, small_df_test = train_test_split(small_df, test_size=0.20, stratify=small_df.speaker_gender, random_state=0)

In [26]:
vec = CountVectorizer(
    ngram_range=(1, 3), 
    stop_words=stop_words,
)  

X_train = vec.fit_transform(small_df_train.clean_text)
X_test = vec.transform(small_df_test.clean_text)

y_train = small_df_train.speaker_gender
y_test = small_df_test.speaker_gender

In [28]:
import pickle

# with open('C:/Users/Ana/OneDrive - Hochschule Düsseldorf/MA/x_train.pkl', 'wb') as f:
#     pickle.dump(X_train, f)

# with open('C:/Users/Ana/OneDrive - Hochschule Düsseldorf/MA/x_test.pkl', 'wb') as f:
#     pickle.dump(X_test, f)

# load
with open('C:/Users/Ana/OneDrive - Hochschule Düsseldorf/MA/x_train.pkl', 'rb') as f:
   df_x_train = pickle.load(f)

with open('C:/Users/Ana/OneDrive - Hochschule Düsseldorf/MA/x_test.pkl', 'rb') as f:
   df_x_test = pickle.load(f)

In [31]:
vec = CountVectorizer(
    ngram_range=(1, 3), 
    stop_words=stop_words,
)  

# X_train = vec.fit_transform(df_train.clean_text)
# X_test = vec.transform(df_test.clean_text)

y_train = df_train.speaker_gender
y_test = df_test.speaker_gender

In [33]:
# alle Daten
nb = MultinomialNB()
nb.fit(df_x_train, y_train)

preds = nb.predict(df_x_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

      female       0.20      0.00      0.00     45513
        male       0.77      1.00      0.87    148740

    accuracy                           0.77    194253
   macro avg       0.48      0.50      0.43    194253
weighted avg       0.63      0.77      0.66    194253



In [34]:
joblib.dump(nb, "nb.joblib")
joblib.dump(vec, "vec.joblib")

['vec.joblib']

In [27]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

preds = nb.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

      female       0.89      0.01      0.02      1433
        male       0.93      1.00      0.96     18567

    accuracy                           0.93     20000
   macro avg       0.91      0.51      0.49     20000
weighted avg       0.93      0.93      0.90     20000



In [63]:
# gender in Zahlen umwandeln TODO
df["speaker_gender"] = pd.get_dummies(df["speaker_gender"])["male"]
df.head()

,speaker_gender,text,clean_text
0,False,Sehr geehrter Herr Alterspräsident! So muss ...,sehr geehrter herr alterspräsident so muss ich...
1,True,Herr Präsident! Liebe Kolleginnen und Kolleg...,herr präsident liebe kolleginnen und kollegen ...
2,False,Sehr geehrter Herr Alterspräsident Wolfgang ...,sehr geehrter herr alterspräsident wolfgang sc...
3,True,Sehr geehrter Herr Präsident! Meine lieben K...,sehr geehrter herr präsident meine lieben koll...
4,True,Herr Alterspräsident! Lassen Sie mich zunäch...,herr alterspräsident lassen sie mich zunächst ...
